## ~ 17:15

## 다음 문제를 풀이하시오.

diabetes.csv 파일을 대상으로 다음 문제를 풀이하시오.

In [4]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("diabetes.csv")
df.head(2)

Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0

### Q1. 당뇨 여부인 Outcome과 임신 횟수인 Pregnancies을 제외한 수치형 변수는  결측치가 0으로 기록이 되어 있다. 모든 변수의 결측치 개수를 더한 개수를 기술하시오.
(정답 예시: 123)

In [3]:
df_na = df.iloc[:, 1:-1]
df_na.head(1)

Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0      148             72             35        0  33.6   

   DiabetesPedigreeFunction  Age  
0                     0.627   50

In [6]:
ser = pd.Series([0, 2, 3, 0, 2])

In [7]:
np.where(ser == 0, np.nan, ser)

array([nan,  2.,  3., nan,  2.])

In [9]:
ser.replace(to_replace = 0, value = np.nan)

0    NaN
1    2.0
2    3.0
3    NaN
4    2.0
dtype: float64

In [11]:
sum(df_na["Glucose"] == 0)

5

In [10]:
df_na.apply(lambda x: sum(x == 0))

Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
dtype: int64

In [12]:
df_na.apply(lambda x: sum(x == 0)).sum()

652

### Q2. Outcome 변수를 제외한 변수간 상관분석을 실시하고 상관계수가 가장 높은 조합의 변수명과 상관계수를 기술하시오.
 - 변수명은 알파벳 순서대로 기술한다.
 - 상관계수는 반올림하여 소수점 셋째자리까지 기술한다.
 
(정답 예시: aaa, bbb, 0.123)

In [15]:
df_q2 = df.drop("Outcome", axis = 1).copy()
df_q2.head(1)

Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   

   DiabetesPedigreeFunction  Age  
0                     0.627   50

In [ ]:
df_q2.corr().round(2)

In [19]:
df_q2_corr = df_q2.corr()
type(df_q2_corr)

pandas.core.frame.DataFrame

In [20]:
df_q2_corr.apply(lambda x: x[x != 1].max())

Pregnancies                 0.544341
Glucose                     0.331357
BloodPressure               0.281805
SkinThickness               0.436783
Insulin                     0.436783
BMI                         0.392573
DiabetesPedigreeFunction    0.185071
Age                         0.544341
dtype: float64

In [21]:
df_q2_corr.apply(lambda x: x[x != 1].idxmax())

Pregnancies                           Age
Glucose                           Insulin
BloodPressure                         BMI
SkinThickness                     Insulin
Insulin                     SkinThickness
BMI                         SkinThickness
DiabetesPedigreeFunction          Insulin
Age                           Pregnancies
dtype: object

In [ ]:
df_q2_corr_max = pd.DataFrame({"col":  df_q2_corr.apply(lambda x: x[x != 1].idxmax()),
                               "corr": df_q2_corr.apply(lambda x: x[x != 1].max())})
df_q2_corr_max = df_q2_corr_max.reset_index()
df_q2_corr_max

In [25]:
df_q2_corr_max.loc[df_q2_corr_max["corr"] == df_q2_corr_max["corr"].max(), ]

index          col      corr
0  Pregnancies          Age  0.544341
7          Age  Pregnancies  0.544341

### Q3. 임신 경험 여부와 BMI간의 관계를 보고자 한다. 이때 적절한 검정방법을 사용하고 해당 검정의 검정통계량의 절대값을 반올림하여 소수점 셋째 자리 까지 표기하시오.
(정답 예시: 1.234)

In [26]:
df_q3 = df.loc[:, ["Pregnancies", "BMI"]].copy()
df_q3.head(2)

Pregnancies   BMI
0            6  33.6
1            1  26.6

In [27]:
from scipy.stats import ttest_ind

In [28]:
stat, p = ttest_ind(df_q3.loc[df_q3["Pregnancies"] == 0, "BMI"],
                    df_q3.loc[df_q3["Pregnancies"] >  0, "BMI"])
round(abs(stat), 3)

3.341

### Q4. 임신 횟수, 혈당, 혈압, 피부 두께로 BMI를 설명할 수 있는지 확인해보려 한다. BMI를 종속변수로 한 다중선형 회귀분석을 실시하고 유의미한 변수의 개수와 결정계수를 기술하시오.
 - 결측치는 모두 제거하고 분석을 실시한다.
 - 결정계수는 소수점 둘 째 자리까지 기술한다.

(정답 예시: 1, 0.12)

In [29]:
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [31]:
df_q4 = df.loc[:, ["Pregnancies", "Glucose", "BloodPressure",
                   "SkinThickness", "BMI"]].copy()
df_q4.head(2)

Pregnancies  Glucose  BloodPressure  SkinThickness   BMI
0            6      148             72             35  33.6
1            1       85             66             29  26.6

In [32]:
from statsmodels.formula.api import ols

In [34]:
formula = "BMI ~ " + " + ".join(df_q4.columns[:-1])
formula

'BMI ~ Pregnancies + Glucose + BloodPressure + SkinThickness'

In [35]:
model = ols(formula, data = df_q4).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    BMI   R-squared:                       0.225
Model:                            OLS   Adj. R-squared:                  0.221
Method:                 Least Squares   F-statistic:                     55.50
Date:                Thu, 27 Aug 2020   Prob (F-statistic):           4.23e-41
Time:                        17:44:55   Log-Likelihood:                -2577.0
No. Observations:                 768   AIC:                             5164.
Df Residuals:                     763   BIC:                             5187.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        18.1608      1.251     14.514      0.000      15.704      20.617
Pregnancies      -0.0064      0.076     -0.083      0.934      -0.156       0.143
Glucose           0.0428      0.008      5.346      0.000       0.027       0.059
BloodPressure     0.0751      0.014      5.541      0.000       0.048       0.102
SkinThickness     0.1701      0.016     10.489      0.000       0.138       0.202
==============================================================================
Omnibus:                       59.054   Durbin-Watson:                   1.912
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              274.126
Skew:                          -0.100   Prob(JB):                     2.98e-60
Kurtosis:                       5.920   Cond. No.                         718.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Q5. 결측치 처리에 따른 효과를 알아보고자 한다. 4번에서 사용된 독립변수 중 결측치를 모두 해당 변수의 평균치로 치환하고 다중선형 회귀분석을 실시하여 얻은 결정계수와 결측치를 연령대 평균 값으로 치환한 후 다중선형 회귀분석을 실시하여 얻은 결정계수를 차례대로 기술하시오.
 - 단, 이 분석은 70세 미만의 데이터만 사용한다.
 - 연령대 구분 단위는 10살 단위로 한다(17 => 1, 30 => 3, 45 => 4)
 - 결정계수는 소수점 둘 째 자리까지 기술한다.

(정답 예시: 0.12, 0.34)

### Q6. 분류모델의 성능을 비교하고자 한다. Outcome 변수를 독립변수로 하고 나머지 변수를 독립변수로 하고, 독립변수의 경우 결측치로 간주되는 것은 모두 제거한다. 이항 로지스틱 회귀분석과, 의사결정나무로 학습을 하고 그 정확도를 각각 기술하시오.
 - 학습은 최초 100개 데이터를 제외한 나머지로 실시한다.
 - 평가는 최초 100개 데이터로 실시한다.
 - 이항 로지스틱 회귀분석의 경우 유의미 하지 않은 변수는 제거한다.
 - 이항 로지스틱 회귀분석의 문턱값은 0.5로 설정한다.
 - 정확도는 반올림하여 소수점 셋째 자리까지 표기한다.
 
(정답 예시: 0.123, 0.456)